In [ ]:
import pandas as pd
import numpy as np
properties = pd.read_csv('../matrix_properties/processed_properties.csv', header=0, index_col=0)
properties

In [ ]:
time_files = ['../processed_timings/system_specific/bridges_all_np_timings_processed.csv',
            '../processed_timings/system_specific/comet_all_np_timings_processed.csv',
            '../processed_timings/system_specific/laptop_all_np_timings_processed.csv',
            '../processed_timings/system_specific/summit_all_np_timings_processed.csv',
            '../processed_timings/system_specific/stampede_all_np_timings_processed.csv']
times_array = []
for t in time_files:
    times_array.append(pd.read_csv(t, header=0, index_col=0))
combined_times = pd.concat(times_array)
combined_times = combined_times.drop(labels=['system', 'solver', 'prec', 'status',
                                            'new_time', 'good_or_bad', 'resid', 'iters'],
                                         axis=1)
combined_times = combined_times.drop_duplicates()
combined_times

In [ ]:
systems_info = pd.read_csv('../systems_info/systems_info.csv')
systems_info.system_id = systems_info.system_id.astype(int)
systems_info = systems_info.drop(labels=['system'], axis=1)
systems_info

In [ ]:
pt = pd.merge(properties, combined_times, on='matrix_id')
pts = pd.merge(systems_info, pt, on='system_id')
pts = pts.dropna()

pts = pts.drop(
        labels=['matrix_y', 'matrix_x'], axis=1)

In [ ]:
pts.status_id.unique()

In [ ]:
# Makes label for error
lbl_error = []
i = 0
for row in pts.itertuples():
    if getattr(row, "status_id") == -1:
        lbl_error.append(1)
    else:
        lbl_error.append(0)
lbl_error

In [ ]:
# Makes label for convergence
lbl_converged = []
i = 0
for row in pts.itertuples():
    if getattr(row, "status_id") == 1:
        lbl_converged.append(1)
    else:
        lbl_converged.append(0)
lbl_converged

In [ ]:
grouped = pts.groupby(['matrix_id', 'system_id', 'np','status_id'])
best_times = grouped['time'].aggregate(np.min)
best_times

In [ ]:
# Finds the best times for each specific combination of np and sys
lbl_best_at_specific_sys_and_np = []
i = 0
for row in pts.itertuples():
    cur_matrix_id = getattr(row, 'matrix_id')
    cur_system_id = getattr(row, 'system_id')
    cur_np = getattr(row, 'np')
    cur_status_id = getattr(row, 'status_id')
    try:
        print(best_times[cur_matrix_id][cur_system_id][cur_np][1])
    except:
        print("error: 0")